### MCP Server Types

This notebook contains simplr projects of using MCP servers - locally hosted and online MCP servers.

In [ ]:
#importing required libraries
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from agents.extensions.models.litellm_model import LitellmModel

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from datetime import datetime

In [ ]:
load_dotenv(override=True)

In [ ]:
api_key = os.getenv("AMAZON_BEDROCK_API_KEY")

In [ ]:
model_name = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model=LitellmModel(model=model_name, api_key=api_key)

#### The first type of MCP Server: runs locally.

This is a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [ ]:
env = {
    "LIBSQL_URL": "file:./memory/ian.db",
    "PATH": "/usr/bin/node" + os.environ["PATH"]
}
params = {
    "command": "npx",
    "args": ["-y", "mcp-memory-libsql"],
    "env": env,
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Ian. I'm an DevOps eng currently learning LLM engineering. I'm currently learning MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
        name="simple_agent",
        instructions=instructions,
        model=model,
        mcp_servers=[mcp_server]
    )

    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
memory_test_prompt = "My name's Ian. What do you know about me?"

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, memory_test_prompt)
    display(Markdown(result.final_output))

#### Remote MCP Server

A remote MCP server is also called hosted MCP server or managed MCP server.

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


#### polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

Check up on docs and pricing:

https://polygon.io

Generate api key and store in .env file.

`POLYGON_API_KEY=xxxx`

In [ ]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [ ]:
from polygon import RESTClient

client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

#### Python module that caches end of day prices

Python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.

In [ ]:
from market import get_share_price

get_share_price("AAPL")

In [ ]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

#### Market MCP Server

Just as we did with accounts.py; see `market_server.py`

In [ ]:
params = {
    "command": "uv", 
    "args": ["run", "market_server.py"]
    }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

In [ ]:
instructions = "You answer questions about the stock market."
request = "How is apple performing right now compared to amazon?"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(
        name="market_agent", 
        instructions=instructions, 
        model=model, 
        mcp_servers=[mcp_server]
    )
    
    with trace("share_price_conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))